# Plot of the data spans

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys
import plotly.graph_objects as go

import open3d as o3d

cwd = os.getcwd()
base_dir = os.path.dirname(os.path.dirname(cwd))
navion_data_dir = base_dir + "/data/navion_data/data"
src_dir = base_dir + "/src"
plot_dir = cwd + "/plots/"

sys.path.insert(0, src_dir)
from paper import latex_utils
from data_analysis import load_navion_format, convert_frames

In [ ]:
df_navion = convert_frames(load_navion_format(navion_data_dir))

# Apply navion coordinate transform
df_navion["x"] = -df_navion["x"]
df_navion["y"] = -df_navion["y"] + 0.2

In [ ]:
def alpha_shape_surface_open3d(
    df: "pd.DataFrame",
    alpha: float = 1.5,
    max_points: int = 60_000,
    res: float = 0.0016,
    jitter_frac: float = 1e-7,
    opacity: float = 0.25,
    show_points: bool = True,
    points_sample: int = 15_000,
    point_size: float = 1.5,
    point_opacity: float = 0.08,
    point_color: str = "blue",          # "red" or "blue"
    print_counts: bool = True,
    view_eye=(1.6, 1.6, 0.9),
    store_to: str = None,
    hide_axes: bool = True,             # backwards compatibility
    show_axes: bool = None,             # True/False; overrides hide_axes if not None
    transparent: bool = True,
    png_width: int = 1000,
    png_height: int = 1000,
    png_scale: int = 4,
    range_pad_frac: float = 0.02,
    latex_params: dict = None,          # e.g. latex_prms_singlecol / latex_prms_2img / latex_prms_3img
    usetex: bool = True,
):
    

    latex_params = {} if latex_params is None else dict(latex_params)
    latex_params.setdefault("usetex", usetex)

    unit_scale = 100.0  # m -> cm

    def _ticks_every_10cm(lo: float, hi: float, step: float = 10.0):
        """
        Build tick positions in [lo, hi] at multiples of `step` (cm).
        Robust to negative ranges and non-integer bounds.
        """
        if not (np.isfinite(lo) and np.isfinite(hi)):
            return None
        if hi < lo:
            lo, hi = hi, lo
        start = np.ceil(lo / step) * step
        end = np.floor(hi / step) * step
        if end < start:
            return None
        # + step/2 for floating safety in arange endpoint
        return np.arange(start, end + 0.5 * step, step)

    with latex_utils.rc_context_latex(**latex_params):
        if res is None or res <= 0:
            raise ValueError("res must be a positive float (e.g. 0.0016).")

        point_color = str(point_color).lower().strip()
        if point_color not in {"red", "blue"}:
            raise ValueError('point_color must be "red" or "blue".')

        if show_axes is not None:
            hide_axes = not bool(show_axes)

        if view_eye is not None:
            eye_arr = np.asarray(view_eye, dtype=float).reshape(3,)
            if not np.all(np.isfinite(eye_arr)):
                raise ValueError("view_eye must be a finite 3D iterable, e.g. (1.6, 1.6, 0.9).")
            eye = dict(x=float(eye_arr[0]), y=float(eye_arr[1]), z=float(eye_arr[2]))
        else:
            eye = None

        # --- pull xyz (meters) ---
        P_all_m = df[["x", "y", "z"]].apply(pd.to_numeric, errors="coerce").dropna().to_numpy()
        n_total = len(P_all_m)
        if n_total < 100:
            raise ValueError("Not enough points for a surface.")

        # --- convert to cm for everything downstream ---
        P_all = P_all_m * unit_scale  # cm

        # ---- locked bounds from all points (cm) ----
        mins_all = P_all.min(axis=0)
        maxs_all = P_all.max(axis=0)
        spans = np.maximum(maxs_all - mins_all, 1e-12)
        pad = spans * float(range_pad_frac)

        xr = [float(mins_all[0] - pad[0]), float(maxs_all[0] + pad[0])]
        yr = [float(mins_all[1] - pad[1]), float(maxs_all[1] + pad[1])]
        zr = [float(mins_all[2] - pad[2]), float(maxs_all[2] + pad[2])]

        # Manual aspect ratio (normalized)
        dx, dy, dz = spans
        m = float(max(dx, dy, dz))
        aspectratio = dict(x=float(dx / m), y=float(dy / m), z=float(dz / m))

        # --- downsample for mesh building ---
        P = P_all
        if len(P) > max_points:
            rng = np.random.default_rng(0)
            P = P[rng.choice(len(P), size=max_points, replace=False)]

        # --- merge points by resolution ---
        res_cm = float(res) * unit_scale
        key = np.round(P / res_cm).astype(np.int64)
        _, keep = np.unique(key, axis=0, return_index=True)
        P = P[np.sort(keep)]
        n_unique_res = len(P)

        if print_counts:
            print(
                f"total points: {n_total:,} | unique@res={res_cm:g} cm "
                f"(from {res:g} m): {n_unique_res:,} | used_for_mesh: {len(P):,}"
            )

        # --- normalize scale for alpha-shape stability ---
        mins = P.min(axis=0)
        maxs = P.max(axis=0)
        center = (mins + maxs) / 2
        diag = np.linalg.norm(maxs - mins)
        diag = max(diag, 1e-12)
        Pn = (P - center) / diag

        # --- tiny jitter ---
        rng = np.random.default_rng(1)
        Pn = Pn + rng.normal(scale=jitter_frac, size=Pn.shape)

        pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(Pn))

        mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
        mesh.remove_duplicated_triangles()
        mesh.remove_degenerate_triangles()
        mesh.remove_unreferenced_vertices()
        mesh.compute_vertex_normals()

        V = np.asarray(mesh.vertices)
        T = np.asarray(mesh.triangles)

        # scale back to original coordinates (cm)
        V = V * diag + center

        fig = go.Figure()

        fig.add_trace(
            go.Mesh3d(
                x=V[:, 0], y=V[:, 1], z=V[:, 2],
                i=T[:, 0], j=T[:, 1], k=T[:, 2],
                opacity=opacity,
                hoverinfo="skip",
                name="surface",
            )
        )

        if show_points:
            Q = P_all
            if len(Q) > points_sample:
                rng = np.random.default_rng(2)
                Q = Q[rng.choice(len(Q), size=points_sample, replace=False)]

            qkey = np.round(Q / res_cm).astype(np.int64)
            _, qkeep = np.unique(qkey, axis=0, return_index=True)
            Q = Q[np.sort(qkeep)]

            fig.add_trace(
                go.Scatter3d(
                    x=Q[:, 0], y=Q[:, 1], z=Q[:, 2],
                    mode="markers",
                    marker=dict(size=point_size, opacity=point_opacity, color=point_color),
                    hoverinfo="skip",
                    name="points",
                )
            )

        bg = "rgba(0,0,0,0)" if transparent else "white"

        # Axis labels
        xlab = "x (cm)"
        ylab = "y (cm)"
        zlab = "z (cm)"

        # --- y ticks every 10 cm (matching x) ---
        xticks_10 = _ticks_every_10cm(xr[0], xr[1], step=10.0)
        yticks_10 = _ticks_every_10cm(yr[0], yr[1], step=10.0)

        if hide_axes:
            axis_off = dict(
                visible=False,
                showbackground=False,
                showgrid=False,
                zeroline=False,
                showline=False,
                ticks="",
                showticklabels=False,
                title="",
            )
            xax = {**axis_off, "range": xr}
            yax = {**axis_off, "range": yr}
            zax = {**axis_off, "range": zr}
        else:
            axis_on = dict(
                visible=True,
                range=None,
                showbackground=False,
                showgrid=False,
                zeroline=False,
                showline=True,
                linecolor="black",
                linewidth=2,
                ticks="outside",
                tickcolor="black",
                tickfont=dict(color="black"),
            )

            # x axis: keep visible, ticks every 10 cm
            xax = {
                **axis_on,
                "range": xr,
                "title": {"text": xlab, "font": {"color": "black"}},
            }
            if xticks_10 is not None:
                xax.update({"tickmode": "array", "tickvals": xticks_10})

            # y axis: keep visible, ticks every 10 cm
            yax = {
                **axis_on,
                "range": yr,
                "title": {"text": ylab, "font": {"color": "black"}},
            }
            if yticks_10 is not None:
                yax.update({"tickmode": "array", "tickvals": yticks_10})

            # --- hide z axis entirely (no line, no ticks, no label) ---
            zax = dict(
                visible=False,
                showbackground=False,
                showgrid=False,
                zeroline=False,
                showline=False,
                ticks="",
                showticklabels=False,
                title="",
                range=zr,  # keep range so the framing/aspect stays consistent
            )

        camera = dict(eye=eye) if eye is not None else None

        fig.update_layout(
            scene=dict(
                bgcolor=bg,
                xaxis=xax,
                yaxis=yax,
                zaxis=zax,
                aspectmode="manual",
                aspectratio=aspectratio,
            ),
            scene_camera_projection_type="orthographic",
            scene_camera=camera,
            paper_bgcolor=bg,
            margin=dict(l=0, r=0, t=0, b=0),
            showlegend=False,
            title="",
        )

        if store_to:
            import plotly.io as pio
            out = store_to if store_to.lower().endswith(".png") else f"{store_to}.png"

            # Ensure directory exists
            os.makedirs(os.path.dirname(out), exist_ok=True)
            pio.write_image(fig, out, format="png", width=png_width, height=png_height, scale=png_scale)
            print(f"Saved PNG: {out} (effective {png_width*png_scale}x{png_height*png_scale}px)")

        return fig

In [ ]:
fig = alpha_shape_surface_open3d(
    df_navion,
    point_color="blue",
    show_axes=True,   # axis visible
    point_opacity=1,
    opacity=0,
    store_to=plot_dir + "navion_dataset.png",
    view_eye=(0.2, 0.2, 1.0),
    latex_params=latex_utils.latex_prms_singlecol,
)
fig.show()